# 1. 데이터를 로딩하세요

In [4]:
import pandas as pd

In [21]:
df = pd.read_csv("cdn_page_load_times.csv")
df.head()


,CDN_A,CDN_B,CDN_C
0,103.115543,102.342433,98.371838
1,94.341060,118.835131,86.193353
2,114.586739,104.359728,79.047870
3,156.155017,90.357875,63.730201
4,156.920524,75.013771,116.655484


# 2. 데이터를 분석하기에 적합한 가설검정 방법을 선택하세요

- 일원분산
셋 이상의 그룹 간 차이가 존재하는지 확인하기 위한 가설 검정 방법 

이유:
독립된 3개의 그룹 (A, B, C)

각 그룹의 데이터는 연속형 (숫자: 로딩 시간)

평균의 차이가 우연인지, 유의미한 차이인지 판단하고 싶음


# 3. 선택한 가설검정 방법을 수행하기 위해 데이터의 전제조건에 부합하는지 확인하고, 결과를 해석하시오 (정규성 검정, 등분산성 검정 등)

In [28]:
from scipy.stats import shapiro

In [ ]:
# 데이터 분할 (결측이 포함되어 있을 수 있으므로 dropna를 사용) > 데이터 정리
# df 데이터 선택 / dropna 결측치 제거 / values 넘파이 배열로 변환
A = df['CDN_A'].dropna().values
B = df['CDN_B'].dropna().values
C = df['CDN_C'].dropna().values

## 등분산성 검정

In [32]:
# stats.levene 분산이 같은지 (등분산) 검사 함수
# levene_stat: 검정통계량
# p-value > 0.05 등분산성 만족 anova 가능 / <0.05 등분산성 깨짐
levene_stat, levene_p_value = stats.levene(A, B, C, center='median')
print(f"levene_stat: {levene_stat}, levene_p_value: {levene_p_value}")

levene_stat: 0.6331383822746123, levene_p_value: 0.5316379878247817


## 정규성 검정

In [37]:
# 정규분포를 따르느냐? p value > 0.05 정규성 만족

shapiro_A = stats.shapiro(A)
shapiro_B = stats.shapiro(B)
shapiro_C = stats.shapiro(C)

print("CDN A 정규성 검정:", shapiro_A)
print("CDN B 정규성 검정:", shapiro_B)
print("CDN C 정규성 검정:", shapiro_C)

CDN A 정규성 검정: ShapiroResult(statistic=np.float64(0.991459163337052), pvalue=np.float64(0.7808083429548649))
CDN B 정규성 검정: ShapiroResult(statistic=np.float64(0.9883888363507614), pvalue=np.float64(0.5373048524875039))
CDN C 정규성 검정: ShapiroResult(statistic=np.float64(0.9882544987868058), pvalue=np.float64(0.5271579491017457))


# 4. 가설검정을 수행하고, 결과 해석

In [41]:
# from scipy import stats
# print(stats.f_oneway(A, B, C))

# 귀무가설: 세 cdn의 평균 페이지 로딩 시간은 모두 같다
# 대립가설: 하나 이상의 cdn은 평균 로딩 시간이 다르다

# p-value가 0.05보다 작다 → 세 그룹 간에 평균 차이가 있음
# 이럴 경우 어디서 차이가 나는지 확인하려면 사후 검정(5번) 으로

# f statistic은 그룹 간 차이가 얼마나 큰지 수치로 보여줌, 값이 클수록 그룹간 차이가 큼!

from scipy import stats

anova_result = stats.f_oneway(A, B, C)
print("F-statistic:", anova_result.statistic)
print("p-value:", anova_result.pvalue)

F-statistic: 39.192607237897704
p-value: 7.842852500131719e-16


# 5. 사후검정 수행, 결과 해석

In [ ]:
from statsmodels.stats import multicomp 
# 분석 대상 데이터 / 데이터를 하나로 합침
Data = A.tolist() + B.tolist() + C.tolist()
# 각 값이 속한 그룹 이름
Group = ['A'] * len(A) + ['B'] * len(B) + ['C'] * len(C)
# len a 안에 몇개의 데이터가 있는지 세는 함수

print(Data)
print(Group)

[103.11554292810683, 94.34105952666415, 114.5867394576332, 156.1550173607268, 156.92052384495878, 104.22755673229447, 107.6140682338326, 99.04850248155746, 148.76962225559578, 120.67875774486048, 124.57925527405276, 142.7696207450541, 123.44878790011826, 126.04694769521006, 87.43917094953854, 110.68542361010286, 151.60296437484098, 144.90959694828035, 65.20273208108758, 143.61211363092735, 138.5591281475343, 82.71200803448637, 97.40372429680264, 149.94563510124112, 110.2327374859087, 139.99874869631796, 123.28201942706104, 161.9196384401359, 136.59208341566415, 116.57533356519107, 133.72621373693235, 129.3428211825181, 95.01722972874438, 124.9823771948719, 161.20014267319868, 113.74100829349514, 146.0151175712553, 133.59273662589052, 140.3102873360546, 102.0955106108504, 104.08087761592591, 114.34448089002932, 126.35485691237244, 132.5564134783298, 157.08690748668684, 107.43391692732573, 118.31356885124347, 124.53691918328128, 63.8475098318646, 122.6300952041994, 111.97994337764383, 12

In [43]:
# 사후 분석
print(multicomp.pairwise_tukeyhsd(Data, Group))

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     A      B -24.8154    0.0 -31.4178 -18.213   True
     A      C -12.5563    0.0 -19.1587 -5.9538   True
     B      C  12.2591 0.0001   5.6567 18.8616   True
-----------------------------------------------------


## 각 항목 설명

- group1, group2	비교한 그룹
- meandiff	group1 - group2의 평균 차이
- p-adj	조정된 p-value (0.05보다 작으면 의미 있음)
- lower, upper	평균 차이의 신뢰구간 (95%)
-  reject	True면 두 그룹은 통계적으로 차이 있음

## 비교 해석
1. A vs B
평균 차이: -24.8154 (즉, A가 B보다 24.8 정도 더 빠름)

p-value: 0.0 

2. A vs C
평균 차이: -12.5563 (A가 C보다도 더 빠름)

p-value: 0.0

3. B vs C
평균 차이: 12.2591 (B가 C보다 느림)

p-value: 0.0001


## 정리 한 줄 요약 
A가 제일 빠르고, 그다음 C, 제일 느린 건 B!
모든 조합에서 통계적으로 유의미한 차이 있음